# Imports

In [ ]:
import os
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint

# Constants

In [ ]:
input_width = 150
input_height = 150

epochs = 20
learning_rate = 0.0001

train_batch_size = 32
validation_batch_size = 32

train_steps = 4
validation_steps = 1

checkpoint_path = "checkpoints/convolutional_neural_network/cp-{epoch:04d}.ckpt"
checkpoint_frequency = 5

train_dir = 'train'
validation_dir = 'validation'

train_0_dir = os.path.join(train_dir, '0')
train_1_dir = os.path.join(train_dir, '1')

validation_0_dir = os.path.join(validation_dir, '0')
validation_1_dir = os.path.join(validation_dir, '1')

# Configuration

### Model Configuration 

The images will go into the model as `150x150` color images. The architecture of the model has three `convolution + ReLU + max-pooling` modules and a fully connected layer stacked on top of each other. Each convolution extracts `3x3` filters, and each max-polling layer uses a `2x2` window. Dropout is added before the final classification layer to reduce overfitting.

In [ ]:
img_input = layers.Input(shape=(input_width, input_height, 3))

# First convolution extracts 16 filters
x = layers.Conv2D(16, 3, activation='relu')(img_input)
x = layers.MaxPooling2D(2)(x)

# Second convolution extracts 32 filters
x = layers.Conv2D(32, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)

# Third convolution extracts 64 filters
x = layers.Conv2D(64, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)

# Flatten feature map in order to add fully connected layers
x = layers.Flatten()(x)

# Create a fully connected layer with ReLU activation and 512 hidden nodes
x = layers.Dense(512, activation='relu')(x)

# Add a dropout rate of 0.2
x = layers.Dropout(0.2)(x)

# Create output layer with a single node and sigmoid activation
output = layers.Dense(1, activation='sigmoid')(x)

# Create model
model = Model(img_input, output)

# Print architecture of model
model.summary()

### Training Configuration

Train model with the `binary_crossentropy` loss function, use the `rmsprop` optimizer, and monitor accuracy during training. Configure checkpoints to save weights of the model.

In [ ]:
# Configure training specifications
model.compile(loss='binary_crossentropy', optimizer=RMSprop(lr=learning_rate), metrics=['acc'])

# Configure checkpoints
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    period=checkpoint_frequency)

# Data Preprocessing

Create data generators to read pictures and feed them to the network. Resize all images to be `150x150` pixels. Normalize data by making pixel values in range `[0, 1]` instead of `[0, 225]`. Augment training images by applying the following random transformations:
- `rotation_range` is for randomly rotating the image.
- `width_shift` and `height_shift` are for randomly translating pictures vertically or horizontally.
- `shear_range` is for randomly applying shearing transformations.
- `zoom_range` is for randomly zooming inside pictures.
- `horizontal_flip` is for randomly flipping half of the images horizontally.
- `fill_mode` is the strategy used for filling in newly created pixels, which can appear after a rotation or a width/height shift.

In [ ]:
# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

val_datagen = ImageDataGenerator(rescale=1./255)

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        train_dir,  # This is the source directory for training images
        target_size=(input_width, input_height),
        batch_size=train_batch_size,
        class_mode='binary')

# Flow validation images in batches of 20 using val_datagen generator
validation_generator = val_datagen.flow_from_directory(
        validation_dir,
        target_size=(input_width, input_height),
        batch_size=validation_batch_size,
        class_mode='binary')

# Training

In [ ]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=train_steps,  # training images = batch_size * steps
      epochs=epochs,
      callbacks=[cp_callback],
      validation_data=validation_generator,
      validation_steps=validation_steps,  # validation images = batch_size * steps
      verbose=2)

# Plot Accuracy and Loss

Plot the training/validation loss and accuracy collected during training.

In [ ]:
# Retrieve a list of accuracy results on training and validation data
# sets for each training epoch
acc = history.history['acc']
val_acc = history.history['val_acc']

# Retrieve a list of list results on training and validation data
# sets for each training epoch
loss = history.history['loss']
val_loss = history.history['val_loss']

# Get number of epochs
epochs = range(len(acc))

%matplotlib inline

# Plot training and validation accuracy per epoch
plt.plot(epochs, acc, label='Training')
plt.plot(epochs, val_acc, label='Validation')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.legend()

plt.figure()

# Plot training and validation loss per epoch
plt.plot(epochs, loss, label='Training')
plt.plot(epochs, val_loss, label='Validation')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.legend()